# All the parameters that need to be changed

In [1]:
# Delaware
state_ab = "de"

## Data
1. Download all the data in directory "il_data"
2. Eextract them all

In [2]:
data_folder = state_ab + "_data/"
population1_data = "./{}{}_pl2020_b/{}_pl2020_p1_b.shp".format(data_folder, state_ab, state_ab)
population2_data = "./{}{}_pl2020_b/{}_pl2020_p2_b.shp".format(data_folder, state_ab, state_ab)
vap_data =  "./{}{}_pl2020_b/{}_pl2020_p4_b.shp".format(data_folder, state_ab, state_ab)
vest20_data = "./{}{}_vest_20/{}_vest_20.shp".format(data_folder, state_ab, state_ab)
vest18_data = "./{}{}_vest_18/{}_vest_18.shp".format(data_folder, state_ab, state_ab)
vest16_data = "./{}{}_vest_16/{}_vest_16.shp".format(data_folder, state_ab, state_ab)
send_data = "./{}{}_sldu_adopted_2022/DESenate22.shp".format(data_folder, state_ab)
hdist_data = "./{}{}_sldl_adopted_2022/March 2022 Redistricting Clean Up File.shp".format(data_folder, state_ab)

## Parameters that needs to be manually checked

### base vest data
start_col = 5\
vest_base_data = vest20\
year = '20'

### district data
district column name of cong_df, send, hdist when calling add_dist()

# Program starts

In [3]:
import pandas as pd
import geopandas as gpd
import maup
import time
from maup import smart_repair
from gerrychain import Graph
import os

maup.progress.enabled = True

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
def do_smart_repair(df):
    # change it to the UTM it needs for smart_repair
    df = df.to_crs(df.estimate_utm_crs())
    df = smart_repair(df)
    
    # check maup doctor again to see if smart repair works
    if maup.doctor(df) == True:
        # change it back to this UTM for this data
        df = df.to_crs('EPSG:4269')
    else:
        raise Exception('maup.doctor failed')
    
    return df

In [6]:
def add_district(dist_df, dist_name, election_df, col_name):
    # check if it needs to be smart_repair
    if maup.doctor(dist_df) != True:
        dist_df = do_smart_repair(dist_df)

    election_df = gpd.GeoDataFrame(election_df, crs="EPSG:4269")

    # assigne the pricincts
    precincts_to_district_assignment = maup.assign(election_df.geometry, dist_df.geometry)
    election_df[dist_name] = precincts_to_district_assignment
    for precinct_index in range(len(election_df)):
        election_df.at[precinct_index, dist_name] = dist_df.at[election_df.at[precinct_index, dist_name], col_name]

    return election_df

In [7]:
def rename(original, year):
    party = original[6]
    if party == 'R' or party == 'D':
        return original[3:6] + year + original[6]
    else:
        return original[3:6] + year + 'O'

In [8]:
def check_population(population, df):
    pop_check = pd.DataFrame({
        'pop_col': pop_col,
        'population_df': population[pop_col].sum(), 
        'vest_base': df[pop_col].sum(),
        'equal': [x == y for x, y in zip(population[pop_col].sum(), df[pop_col].sum())]
    })
    if pop_check['equal'].mean() < 1:
        print(pop_check)
        raise Exception("population doesn't agree")

    else:
        print("population agrees")

In [9]:
def add_vest(vest, df, year, population, start_col):    
     # check if it needs to be smart_repair
    if maup.doctor(vest) != True:
        vest = do_smart_repair(vest)
    
    # rename the columns
    original_col = vest.columns[start_col:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum() # combine all the other party's vote into columns with sufix "O"
    col_name = list(set(new_col))
    col_name.sort()
    
    # make the blocks from precincts by weight
    vest = gpd.GeoDataFrame(vest, crs="EPSG:4269")
    election_in_block = population[["VAP", 'geometry']] # population_df is in block scale
    blocks_to_precincts_assignment = maup.assign(election_in_block.geometry, vest.geometry)
    weights = election_in_block["VAP"] / blocks_to_precincts_assignment.map(election_in_block["VAP"].groupby(blocks_to_precincts_assignment).sum())
    weights = weights.fillna(0)
    prorated = maup.prorate(blocks_to_precincts_assignment, vest[col_name], weights)
    election_in_block[col_name] = prorated
    
    # assign blocks to precincts
    election_in_block = gpd.GeoDataFrame(election_in_block, crs="EPSG:4269")
    df = gpd.GeoDataFrame(df, crs="EPSG:4269")
    block_to_pricinct_assginment = maup.assign(election_in_block.geometry, df.geometry)
    df[col_name] = election_in_block[col_name].groupby(block_to_pricinct_assginment).sum()
    df = df.groupby(level=0, axis=1).sum()
    
    # check if population agrees
    check_population(population, df)
    
    return df

## Read the census data

In [10]:
population1_df = gpd.read_file(population1_data)
population2_df = gpd.read_file(population2_data)
vap_df = gpd.read_file(vap_data)

In [11]:
population2_df = population2_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])
vap_df = vap_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])

In [12]:
population_df = pd.merge(population1_df, population2_df, on='GEOID20')
population_df = pd.merge(population_df, vap_df, on='GEOID20')

In [13]:
rename_dict = {'P0020001': 'TOTPOP', 'P0020002': 'HISP', 'P0020005': 'NH_WHITE', 'P0020006': 'NH_BLACK', 'P0020007': 'NH_AMIN',
               'P0020008': 'NH_ASIAN', 'P0020009': 'NH_NHPI', 'P0020010': 'NH_OTHER', 'P0020011': 'NH_2MORE',
               'P0040001': 'VAP', 'P0040002': 'HVAP', 'P0040005': 'WVAP', 'P0040006': 'BVAP', 'P0040007': 'AMINVAP',
               'P0040008': 'ASIANVAP', 'P0040009': 'NHPIVAP', 'P0040010': 'OTHERVAP', 'P0040011': '2MOREVAP'}

In [14]:
population_df.rename(columns=rename_dict, inplace = True)

In [15]:
population_df['H_WHITE'] = population_df.apply(lambda t: t['P0010003'] - t['NH_WHITE'], 1)
population_df['H_BLACK'] = population_df.apply(lambda t: t['P0010004'] - t['NH_BLACK'], 1)
population_df['H_AMIN'] = population_df.apply(lambda t: t['P0010005'] - t['NH_AMIN'], 1)
population_df['H_ASIAN'] = population_df.apply(lambda t: t['P0010006'] - t['NH_ASIAN'], 1)
population_df['H_NHPI'] = population_df.apply(lambda t: t['P0010007'] - t['NH_NHPI'], 1)
population_df['H_OTHER'] = population_df.apply(lambda t: t['P0010008'] - t['NH_OTHER'], 1)
population_df['H_2MORE'] = population_df.apply(lambda t: t['P0010009'] - t['NH_2MORE'], 1)

# Read the base vest data
Now using it as a "base precinct", but it could be vest 18 or vest 16 if vest 20 is not working

In [16]:
def add_vest_base(vest, start_col, year):
    original_col = vest.columns[start_col:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum()
    vest = gpd.GeoDataFrame(vest, crs="EPSG:4269")
    
    return vest

### Check if vest 20 can be used as base

In [17]:
vest20 = gpd.read_file(vest20_data)

In [20]:
if maup.doctor(vest20) != True:
    vest20 = do_smart_repair(vest20)

100%|█████████████████████████████████████████████████████████████████████████████████| 434/434 [00:01<00:00, 300.18it/s]


### If it is true for maup doctor, we will use it as the base vest data.
Check where the election column starts, this should be the same for all vest data in that state

In [21]:
vest20.columns

Index(['PRECINCT', 'G20PREDBID', 'G20PRERTRU', 'G20PRELJOR', 'G20PREGHAW',
       'G20USSDCOO', 'G20USSRWIT', 'G20USSLFRO', 'G20USSITUR', 'G20HALDROC',
       'G20HALRMUR', 'G20HALLROG', 'G20HALIPUR', 'G20GOVDCAR', 'G20GOVRMUR',
       'G20GOVLMAC', 'G20GOVIDEM', 'G20LTGDHAL', 'G20LTGRHAL', 'G20INSDNAV',
       'G20INSRPIL', 'geometry'],
      dtype='object')

## Parameters that need to be checked

In [22]:
start_col = 1
vest_base_data = vest20
year = '20'

In [23]:
vest_base = add_vest_base(vest_base_data, start_col, year)

In [31]:
# vap and population have the same GEOID20
blocks_to_precincts_assignment = maup.assign(population_df.geometry, vest_base.geometry)

100%|████████████████████████████████████████████████████████████████████████████████| 434/434 [00:00<00:00, 2103.11it/s]


In [26]:
pop_col = ['TOTPOP', 'HISP', 'NH_WHITE', 'NH_BLACK', 'NH_AMIN', 'NH_ASIAN', 'NH_NHPI', 'NH_OTHER', 'NH_2MORE', 'H_WHITE', 'H_BLACK', 'H_AMIN', 'H_ASIAN', 'H_NHPI', 'H_OTHER', 'H_2MORE', 'VAP', 'HVAP', 'WVAP', 'BVAP', 'AMINVAP', 'ASIANVAP', 'NHPIVAP', 'OTHERVAP', '2MOREVAP']

In [33]:
vest_base[pop_col] = population_df[pop_col].groupby(blocks_to_precincts_assignment).sum()

In [34]:
blocks_to_precincts_assignment

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
20193   NaN
20194   NaN
20195   NaN
20196   NaN
20197   NaN
Length: 20198, dtype: float64

In [24]:
election_df = gpd.GeoDataFrame(vest_base, crs="EPSG:4269")

### Check if the population agrees

In [25]:
check_population(population_df, vest_base)

           pop_col  population_df  vest_base  equal
TOTPOP      TOTPOP       39538223        0.0  False
HISP          HISP       15579652        0.0  False
NH_WHITE  NH_WHITE       13714587        0.0  False
NH_BLACK  NH_BLACK        2119286        0.0  False
NH_AMIN    NH_AMIN         156085        0.0  False
NH_ASIAN  NH_ASIAN        5978795        0.0  False
NH_NHPI    NH_NHPI         138167        0.0  False
NH_OTHER  NH_OTHER         223929        0.0  False
NH_2MORE  NH_2MORE        1627722        0.0  False
H_WHITE    H_WHITE        2581535        0.0  False
H_BLACK    H_BLACK         117758        0.0  False
H_AMIN      H_AMIN         474931        0.0  False
H_ASIAN    H_ASIAN         107152        0.0  False
H_NHPI      H_NHPI          19096        0.0  False
H_OTHER    H_OTHER        8146667        0.0  False
H_2MORE    H_2MORE        4132513        0.0  False
VAP            VAP       30827105        0.0  False
HVAP          HVAP       11083294        0.0  False
WVAP        

Exception: population doesn't agree

# Add more vest data

In [38]:
# check the result here
election_df = add_vest(vest20, election_df, '20', population_df, start_col)

100%|████████████████████████████████████████| 2591/2591 [00:09<00:00, 272.79it/s]


There are 6 overlaps.
There are 1 holes.
There are some invalid geometries.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|███████████████████████████████████████| 2871/2871 [00:01<00:00, 1538.50it/s]


Resolving overlaps...
Filling gaps...


Gaps to simplify: 0it [00:00, ?it/s]
Gaps to fill: 0it [00:00, ?it/s]
 93%|█████████████████████████████████████▉   | 2401/2591 [01:07<00:05, 35.71it/s]


GEOSException: TopologyException: Input geom 1 is invalid: Self-intersection at or near point -91.857651072432958 36.061617137261997 at -91.857651072432958 36.061617137261997

In [39]:
vest16 = gpd.read_file(vest16_data)
vest16.columns

Index(['COUNTY_FIP', 'COUNTY_NAM', 'PRECINCT', 'G16PRERTRU', 'G16PREDCLI',
       'G16PRELJOH', 'G16PREOMCM', 'G16PREGSTE', 'G16PREIHED', 'G16PRECCAS',
       'G16PREIKAH', 'G16USSRBOO', 'G16USSDELD', 'G16USSLGIL', 'G16USSOWRI',
       'geometry'],
      dtype='object')

In [40]:
election_df = add_vest(vest16, election_df, '16', population_df, start_col)

100%|████████████████████████████████████████| 2525/2525 [00:09<00:00, 265.14it/s]


There are 109 overlaps.
There are 696 holes.
There are some invalid geometries.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|███████████████████████████████████████| 4226/4226 [00:02<00:00, 1473.60it/s]


Resolving overlaps...
Assigning order 2 pieces...
1 gaps will remain unfilled, because they either are not simply connected or exceed the area threshold.
Filling gaps...


100%|████████████████████████████████████████| 2525/2525 [00:09<00:00, 275.29it/s]


There are 1 holes.


Exception: maup.doctor failed

## Add the district data

In [45]:
cong_df = gpd.read_file(cd_data).to_crs('EPSG:4269')
send = gpd.read_file(send_data).to_crs('EPSG:4269')
hdist = gpd.read_file(hdist_data).to_crs('EPSG:4269')

In [46]:
cong_df.head()

,ID,DISTRICT,DISTRICTN,geometry
0,1,1,1,"POLYGON ((-91.84959 34.09321, -91.84596 34.093..."
1,2,2,2,"POLYGON ((-92.80842 34.57717, -92.80841 34.577..."
2,3,3,3,"POLYGON ((-94.13968 35.12895, -94.14129 35.128..."
3,4,4,4,"POLYGON ((-94.43980 35.16354, -94.43977 35.164..."


In [47]:
send.head()

,objectid,id,district,districtn,st_area(sh,st_length(,geometry
0,36,1,1,1.0,1.098196e+10,781762.354242,"POLYGON ((-91.16666 33.00428, -91.16793 33.004..."
1,37,2,2,2.0,8.782117e+09,673477.622005,"POLYGON ((-92.98220 33.24656, -92.98220 33.246..."
2,38,3,3,3.0,9.699264e+09,678299.140530,"POLYGON ((-93.01723 33.01732, -93.01760 33.017..."
3,39,4,4,4.0,6.782729e+09,580337.546669,"POLYGON ((-94.04314 33.34380, -94.04304 33.345..."
4,40,5,5,5.0,1.250595e+10,733191.551857,"POLYGON ((-93.44803 34.17484, -93.44829 34.174..."


In [48]:
hdist.head()

,ID,DISTRICT,DISTRICTN,geometry
0,1,1,1,"POLYGON ((-90.58449 36.02915, -90.58495 36.029..."
1,2,2,2,"POLYGON ((-91.07348 36.11487, -91.07349 36.114..."
2,3,3,3,"POLYGON ((-92.52913 36.49853, -92.52485 36.498..."
3,4,4,4,"POLYGON ((-92.89072 36.11424, -92.89067 36.117..."
4,5,5,5,"POLYGON ((-92.89072 36.11424, -92.90198 36.114..."


In [49]:
election_df = add_district(cong_df, "CD", election_df, "DISTRICT")

100%|███████████████████████████████████████████████| 4/4 [00:06<00:00,  1.71s/it]


In [50]:
election_df = add_district(send, "SEND", election_df, "district")

100%|█████████████████████████████████████████████| 35/35 [00:09<00:00,  3.61it/s]


In [51]:
election_df = add_district(hdist, "HDIST", election_df, "DISTRICT")

100%|███████████████████████████████████████████| 100/100 [00:12<00:00,  7.99it/s]


### Put the base precinct year after the precinct information column

In [52]:
base_columns = {}
if 'COUNTY_FIP' + year not in election_df.columns:
    base_columns = {
        'COUNTY_FIP':'COUNTY_FIP'+year,
        'COUNTY_NAM':'COUNTY_NAM'+year,
        'PRECINCT':'PRECINCT'+year,
    }
election_df.rename(columns=base_columns, inplace = True)

In [53]:
# reorder the columns
fixed_columns = [
    'COUNTY_FIP'+year,
    'COUNTY_NAM'+year,
    'PRECINCT'+year,
    'CD',
    'SEND',
    'TOTPOP',
    'NH_2MORE',
    'NH_AMIN',
    'NH_ASIAN',
    'NH_BLACK',
    'NH_NHPI',
    'NH_OTHER',
    'NH_WHITE',
    'HISP',
    'H_AMIN',
    'H_ASIAN',
    'H_BLACK',
    'H_NHPI',
    'H_OTHER',
    'H_WHITE',
    'H_2MORE',
    'VAP',
    'HVAP',
    'WVAP',
    'BVAP',
    'AMINVAP',
    'ASIANVAP',
    'NHPIVAP',
    'OTHERVAP',
    '2MOREVAP']

election_columns = [col for col in election_df.columns if col not in fixed_columns]
final_col = fixed_columns + election_columns
election_df = election_df[final_col]

In [54]:
# store the result in directory "il"
os.makedirs("./{}".format(state_ab))
election_df.to_file("./{}/{}.shp".format(state_ab, state_ab))
election_df.to_file('./{}/{}.geojson'.format(state_ab, state_ab), driver='GeoJSON')

# Only do once to build json and read from file when generating ensembles
graph = Graph.from_file("./{}/{}.shp".format(state_ab, state_ab), ignore_errors=True)
graph.to_json("./{}/{}.json".format(state_ab, state_ab))